In [1]:
import pandas as pd
import numpy as np

In [150]:
# 1. 데이터 로드 및 전처리
df = pd.read_csv('data/healthcare.csv', index_col= 0)

In [151]:
# 보험 청구 금액 예측하는 인공지능 개발

In [152]:
df.head(2)

,Age,Gender,BMI,Region,Smoker,NumVisits,InsuranceClaim
0,51.0,Female,NaN,South,No,19.0,70.081987
1,92.0,Female,38.074006,West,Yes,13.0,92.747518


In [153]:
# 타겟 변수와 피처 분리
y = df['InsuranceClaim']
X = df.drop(columns=['InsuranceClaim'])

In [154]:
# 수치형 컬럼 이름 
numerical = ['Age', 'BMI', 'NumVisits' ]

In [155]:
# 카테고리컬 데이터 컬럼 이름
categorical = ['Gender', 'Region', 'Smoker']

In [156]:
# 수치형 데이터의 NaN을, 각 컬럼의 평균값으로 채운다.
# 카테고리컬 데이터는 레이블인코딩, 원핫인코딩 한다.

In [157]:
# 저희는 sklearn의 pipeline 라이브러릴 사용할 것이니까
# Nan 을 채우는것은, fillna 함수 대신에, SimpleImputer 를 사용한다.

In [158]:
# 수치형 컬럼의 NaN을 각 컬럼의 평균으로 채우는 것 
from sklearn.impute import SimpleImputer
# 기본 값은 'mean' 입니다.
imputer = SimpleImputer( )

In [159]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [160]:
label_encoder = OrdinalEncoder()
onehot_encoder = OneHotEncoder()

In [161]:
from sklearn.compose import ColumnTransformer

In [162]:
df['Gender'].nunique()

2

In [163]:
df['Region'].nunique()

4

In [164]:
df['Smoker'].nunique()

2

In [165]:
label_columns = ['Gender', 'Smoker']
onehot_columns = ['Region']
numeric_columns = ['Age', 'BMI', 'NumVisits']

In [166]:
# 각 전처리할 라이브러리 준비
# 해당 라이브러리 적용할 컬럼리스트를 준비한다.

In [167]:
preprocessor = ColumnTransformer( [  
    ('num', imputer , numeric_columns ),
    ('label', label_encoder, label_columns ),
    ('onehot', onehot_encoder, onehot_columns)
   ] )

In [168]:
# 3. 파이브라인(pipe line)생성

In [169]:
from sklearn.ensemble import RandomForestRegressor

In [170]:
regressor = RandomForestRegressor(random_state=42)

In [171]:
from sklearn.pipeline import Pipeline

In [173]:
pipeline = Pipeline(
    steps= [ 
        ('preprocessing', preprocessor), 
        ('modeling', regressor ) 
     ] 
    )

In [174]:
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Age', 'BMI', 'NumVisits']),
                                                 ('label', OrdinalEncoder(),
                                                  ['Gender', 'Smoker']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['Region'])])),
                ('modeling', RandomForestRegressor(random_state=42))])

In [175]:
# 4. 학습 / 테스트 데이터 분리 후 학습

In [176]:
from sklearn.model_selection import train_test_split

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [179]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Age', 'BMI', 'NumVisits']),
                                                 ('label', OrdinalEncoder(),
                                                  ['Gender', 'Smoker']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['Region'])])),
                ('modeling', RandomForestRegressor(random_state=42))])

In [180]:
y_pred = pipeline.predict(X_test)

In [181]:
from sklearn.metrics import mean_squared_error, r2_score

In [182]:
mean_squared_error(y_test, y_pred)

46.980189783960185

In [183]:
r2_score(y_test, y_pred)

0.7521137711976167

In [184]:
# 파이프라인을 이용하면 좋은점!

## 이 파이프라인만 파일로 저장하면, 서비스 배포할때 이 파일만 있으면,
## 예측할 데이터만 그냥 넣어주면, 파이프라인이 내부적으로 데이터 전처리를 해서
## 결과를 알려준다.

In [185]:
import joblib


In [186]:
joblib.dump(pipeline, 'model/healthcare.pkl')

['model/healthcare.pkl']

In [187]:
joblib.load('model/healthcare.pkl')

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Age', 'BMI', 'NumVisits']),
                                                 ('label', OrdinalEncoder(),
                                                  ['Gender', 'Smoker']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['Region'])])),
                ('modeling', RandomForestRegressor(random_state=42))])